In [1]:
import os
path = os.getcwd()
path

'/Users/sophiacofone/school_local/svi_worm_anno/annot'

In [2]:
from skimage import data
import napari

In [3]:
import skimage.measure as meas

In [4]:
import numpy as np
import matplotlib.pyplot as plt

In [5]:
import tifffile

In [6]:
from pycromanager import Dataset

In [7]:
fname = 'bact_mask/round2/wspa_5_21_23_wspa_denoise_proj_tstack_devC1_pos_9_.tiff'

maskname = 'bact_mask/round2/wspa_5_21_23_wspa_denoise_proj_tstack_devC1_pos_9_mask'

In [8]:
image_data = tifffile.imread(fname)
shape = image_data.shape
print(shape)

(14, 3, 2048, 2048)


In [9]:
class mySet:
    def __init__(self,path,name,data):
        self.path = path
        self.name = name
        self.data = data
        self.current_w = None
        self.num_w = data.shape[0]
        self.q = np.arange(self.num_w).tolist()
        
    def get_next_w(self):
        self.current_w = self.q.pop()
        print(self.current_w)
        return self.data[self.current_w]
    
    def save_annot(self,annot_stack):
        tifffile.imsave(os.path.join(self.path,self.name+'_{}.tiff'.format(self.current_w)),annot_stack)
        
        
def open_worm(viewer,wstack):
#     max_val = np.max(wstack[0, ...])
#     min_val = np.min(wstack[0, ...])
#     min_val_g = np.min(wstack[1, ...])
#     min_val_r = np.min(wstack[3, ...])
    viewer.add_image(
        wstack,
        channel_axis=0,
        name=[f"Pos {0}, Channel Brightfield",f"Position {0}, Channel Green", f"Position {0}, Channel Red"],
        colormap=["gray", "green", 'red'],
#         contrast_limits=[[min_val, max_val], [min_val_g, 230], [min_val_r, 176]]
    )
    
def save_open_next(viewer,myS):
    if myS.current_w is None:
        open_worm(viewer,myS.get_next_w())
    else:
        try:
            names = [l.name for l in viewer.layers]
            red_layer = viewer.layers[[i for i,j in enumerate(names) if j == 'red'][0]].data
            green_layer = viewer.layers[[i for i,j in enumerate(names) if j == 'green'][0]].data
        
            stacked_images = np.dstack((green_layer, red_layer)).transpose(2, 0, 1)
            #print(np.sum(red_layer),np.sum(green_layer))
            myS.save_annot(stacked_images)

            viewer.layers.clear()
        except IndexError:
            print('annotation layers not added yet')
            return
        
        try:
            open_worm(viewer,myS.get_next_w())
        except IndexError:
            print('finished anotating set')
            


previous_layer = None  # Variable to store the previous layer

def add_layer(viewer, color):
    global previous_layer

    # Deactivate the previous layer
    if previous_layer is not None:
        previous_layer.visible = False

    # Create a new labels layer
    labels = np.zeros((2048, 2048), dtype=np.uint16)
    anot_layer = viewer.add_labels(labels, name=color, seed=0.2)
    anot_layer.brush_size = 25
    anot_layer.mode = 'paint'

    # Update the previous layer variable
    previous_layer = anot_layer
        


In [11]:
myS = mySet(path,maskname,image_data)

viewer = napari.Viewer()

viewer.bind_key('g', lambda event: add_layer(viewer, 'green'))
viewer.bind_key('r', lambda event: add_layer(viewer, 'red'))
viewer.bind_key('c', lambda event: save_open_next(viewer,myS))


13
